# Airline Assistant using Gemini API for Image and Audio as well - Live ticket prices using Amadeus API

In [ ]:
import os
import json
import random
import string
import base64
import gradio as gr
import pyaudio
import requests
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from google import genai
from google.genai import types

In [ ]:
load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("API Key not found!")
else:
    print("API Key loaded in memory")

In [ ]:
MODEL_GEMINI = 'gemini-2.5-flash'
MODEL_GEMINI_IMAGE = 'gemini-2.0-flash-preview-image-generation'
MODEL_GEMINI_SPEECH = 'gemini-2.5-flash-preview-tts'

In [ ]:
try:
    client = genai.Client(api_key=api_key)
    print("Google GenAI Client initialized successfully!")
except Exception as e:
    print(f"Error initializing GenAI Client: {e}")
    print("Ensure your GOOGLE_API_KEY is correctly set as an environment variable.")
    exit() 

## Image Generation 

In [ ]:
def fetch_image(city):
    prompt = (
        f"A high-quality, photo-realistic image of a vacation in {city}, "
        f"showing iconic landmarks, cultural attractions, authentic street life, and local cuisine. "
        f"Capture natural lighting, real people enjoying travel experiences, and the unique vibe of {city}'s atmosphere. "
        f"The composition should feel immersive, warm, and visually rich, as if taken by a travel photographer."
)

    response = client.models.generate_content(
        model = MODEL_GEMINI_IMAGE,
        contents = prompt,
        config=types.GenerateContentConfig(
            response_modalities=['TEXT', 'IMAGE']
        )
    )

    for part in response.candidates[0].content.parts:
        if part.inline_data is not None:
            image_data = BytesIO(part.inline_data.data)
            return Image.open(image_data)

    raise ValueError("No image found in Gemini response.")
    

In [ ]:
fetch_image("london")

## Speech Generation

In [ ]:
"""
Kore -- Firm
Puck -- Upbeat
Leda -- Youthful
Iapetus -- Clear
Erinome -- Clear
Sadachbia -- Lively
Sulafat -- Warm
Despina -- Smooth
"""

def talk(message:str, voice_name:str="Leda", mood:str="cheerfully"):
    prompt = f"Say {mood}: {message}"
    response = client.models.generate_content(
        model = MODEL_GEMINI_SPEECH,
        contents = prompt,
        config=types.GenerateContentConfig(
            response_modalities=["AUDIO"],
            speech_config=types.SpeechConfig(
                voice_config=types.VoiceConfig(
                    prebuilt_voice_config=types.PrebuiltVoiceConfig(
                        voice_name=voice_name,
                    )
                )
            ),        
        )
    )

    # Fetch the audio bytes
    pcm_data = response.candidates[0].content.parts[0].inline_data.data
    # Play the audio using PyAudio
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=24000, output=True)
    stream.write(pcm_data)
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Play using simpleaudio (16-bit PCM, mono, 24kHz)
    # play_obj = sa.play_buffer(pcm_data, num_channels=1, bytes_per_sample=2, sample_rate=24000)
    # play_obj.wait_done()  

In [ ]:
talk("Hi, How are you? Welcome to FlyJumbo Airlines","Kore","helpful")

## Ticket Price Tool Function - Using Amadeus API 

In [ ]:
client_id = os.getenv("AMADEUS_CLIENT_ID")
client_secret = os.getenv("AMADEUS_CLIENT_SECRET")

In [ ]:
# Get the token first
def get_amadeus_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }
        
    try:
        response = requests.post(url, headers=headers, data=data, timeout=10)
        response.raise_for_status()
        return response.json()["access_token"]
    
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error {response.status_code}: {response.text}")
    
    except requests.exceptions.RequestException as e:
        print("Network or connection error:", e)
    
    return None

In [ ]:
def get_airline_name(code, token):
    url = f"https://test.api.amadeus.com/v1/reference-data/airlines"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"airlineCodes": code}

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()

    if "data" in data and data["data"]:
        return data["data"][0].get("businessName", code)
    return code

In [ ]:
COMMON_CITY_CODES = {
    "delhi": "DEL",
    "mumbai": "BOM",
    "chennai": "MAA",
    "kolkata": "CCU",
    "bengaluru": "BLR",
    "hyderabad": "HYD",
    "patna": "PAT",
    "raipur": "RPR",
    "panaji": "GOI",
    "chandigarh": "IXC",
    "srinagar": "SXR",
    "ranchi": "IXR",
    "bengaluru": "BLR",
    "thiruvananthapuram": "TRV",
    "bhopal": "BHO",
    "mumbai": "BOM",
    "imphal": "IMF",
    "aizawl": "AJL",
    "bhubaneswar": "BBI",
    "jaipur": "JAI",
    "chennai": "MAA",
    "hyderabad": "HYD",
    "agartala": "IXA",
    "lucknow": "LKO",
    "dehradun": "DED",
    "kolkata": "CCU",

    # Union territories
    "port blair": "IXZ",
    "leh": "IXL",
    "puducherry": "PNY",

    # Major metro cities (for redundancy)
    "ahmedabad": "AMD",
    "surat": "STV",
    "coimbatore": "CJB",
    "vizag": "VTZ",
    "vijayawada": "VGA",
    "nagpur": "NAG",
    "indore": "IDR",
    "kanpur": "KNU",
    "varanasi": "VNS"
}


In [ ]:
city_code_cache = {}

def get_city_code(city_name, token):
    city_name = city_name.strip().lower()

    if city_name in city_code_cache:
        return city_code_cache[city_name]

    if city_name in COMMON_CITY_CODES:
        return COMMON_CITY_CODES[city_name]

    base_url = "https://test.api.amadeus.com/v1/reference-data/locations"
    headers = {"Authorization": f"Bearer {token}"}

    for subtype in ["CITY", "AIRPORT,CITY"]:
        params = {"keyword": city_name, "subType": subtype}
        try:
            response = requests.get(base_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if "data" in data and data["data"]:
                code = data["data"][0]["iataCode"]
                print(f"[INFO] Found {subtype} match for '{city_name}': {code}")
                city_code_cache[city_name] = code
                return code
        except Exception as e:
            print(f"[ERROR] Location lookup failed for {subtype}: {e}")

    return None

In [ ]:
# Getting live ticket price 

def get_live_ticket_prices(origin, destination, departure_date, return_date=None):
    token = get_amadeus_token()

    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {"Authorization": f"Bearer {token}"}

    origin_code =  get_city_code(origin,token)
    destination_code = get_city_code(destination,token)

    if not origin_code:
        return f"Sorry, I couldn't find the airport code for the city '{origin}'."
    if not destination_code:
        return f"Sorry, I couldn't find the airport code for the city '{destination}'."

    params = {
        "originLocationCode": origin_code.upper(),
        "destinationLocationCode": destination_code.upper(),
        "departureDate": departure_date,
        "adults": 1,
        "currencyCode": "USD",
        "max": 1,
    }

    if return_date:
        params["returnDate"] = return_date

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if "data" in data and data["data"]:
            offer = data["data"][0]
            price = offer["price"]["total"]
            airline_codes = offer.get("validatingAirlineCodes", [])
            airline_code = airline_codes[0] if airline_codes else "Unknown"

            try:
                airline_name = get_airline_name(airline_code, token) if airline_code != "Unknown" else "Unknown Airline"
                if not airline_name:  
                    airline_name = airline_code
            except Exception:
                airline_name = airline_code
            
            
            if return_date:
                return (
                    f"Round-trip flight from {origin.capitalize()} to {destination.capitalize()}:\n"
                    f"- Departing: {departure_date}\n"
                    f"- Returning: {return_date}\n"
                    f"- Airline: {airline_name}\n"
                    f"- Price: ${price}"
                )
            else:
                return (
                    f"One-way flight from {origin.capitalize()} to {destination.capitalize()} on {departure_date}:\n"
                    f"- Airline: {airline_name}\n"
                    f"- Price: ${price}"
                )
        else:
            return f"No flights found from {origin.capitalize()} to {destination.capitalize()} on {departure_date}."
    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching flight data: {str(e)}"  
    

In [ ]:
get_live_ticket_prices("london", "chennai", "2025-07-01","2025-07-10")

## Ticket Booking Tool Function - DUMMY

In [ ]:
def book_flight(origin, destination, departure_date, return_date=None, airline="Selected Airline", passenger_name="Guest"):
    # Generate a dummy ticket reference (PNR)
    ticket_ref = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

    # Build confirmation message
    confirmation = (
        f"🎫 Booking confirmed for {passenger_name}!\n"
        f"From: {origin.capitalize()} → To: {destination.capitalize()}\n"
        f"Departure: {departure_date}"
    )

    if return_date:
        confirmation += f"\nReturn: {return_date}"

    confirmation += (
        f"\nAirline: {airline}\n"
        f"PNR: {ticket_ref}\n"
        f"✅ Your ticket has been booked successfully. Safe travels!"
    )

    return confirmation


In [ ]:
print(book_flight("chennai", "delhi", "2025-07-01", "2025-07-10", "Air India", "Ravi Kumar"))

## Gemini Chat Workings

In [ ]:
ticket_price_function_declaration = {
    "name":"get_live_ticket_prices",
    "description": "Get live flight ticket prices between two cities for a given date (round-trip or one-way).\
    The destination may be a city or country (e.g., 'China'). Call this function whenever a customer asks about ticket prices., such as 'How much is a ticket to Paris?'",
    "parameters":{
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Name of the origin city. Example: 'Delhi'",
            },
             "destination": {
                "type": "string",
                "description":"Name of the destination city. Example: 'London'",
            },
            "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format. Example: '2025-07-01'",
            },
            "return_date": {
                "type": "string",
                "description": "Optional return date for round-trip in YYYY-MM-DD format. Leave blank for one-way trips.",
            },
        },
        "required": ["origin", "destination", "departure_date"],
    }
}

In [ ]:
book_flight_function_declaration = {
    "name": "book_flight",
    "description": "Book a flight for the user after showing the ticket details and confirming the booking. "
                   "Call this function when the user says things like 'yes', 'book it', or 'I want to book this flight'.",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Name of the origin city. Example: 'Chennai'",
            },
            "destination": {
                "type": "string",
                "description": "Name of the destination city. Example: 'London'",
            },
            "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format. Example: '2025-07-01'",
            },
            "return_date": {
                "type": "string",
                "description": "Optional return date for round-trip in YYYY-MM-DD format. Leave blank for one-way trips.",
            },
            "airline": {
                "type": "string",
                "description": "Airline name or code that the user wants to book with. Example: 'Air India'",
            },
            "passenger_name": {
                "type": "string",
                "description": "Full name of the passenger for the booking. Example: 'Ravi Kumar'",
            }
        },
        "required": ["origin", "destination", "departure_date", "passenger_name"],
    }
}


In [ ]:
# System Definitions

system_instruction_prompt = (
    "You are a helpful and courteous AI assistant for an airline company called FlyJumbo. "
    "When a user starts a new conversation, greet them with: 'Hi there, welcome to FlyJumbo! How can I help you?'. "
    "Do not repeat this greeting in follow-up messages. "
    "Use the available tools if a user asks about ticket prices. "
    "Ask follow-up questions to gather all necessary information before calling a function."
    "After calling a tool, always continue the conversation by summarizing the result and asking the user the next relevant question (e.g., if they want to proceed with a booking)."
    "If you do not know the answer and no tool can help, respond politely that you are unable to help with the request. "
    "Answer concisely in one sentence."
)

In [ ]:
tools = types.Tool(function_declarations=[ticket_price_function_declaration,book_flight_function_declaration])
generate_content_config = types.GenerateContentConfig(system_instruction=system_instruction_prompt, tools=[tools])

In [ ]:
def handle_tool_call(function_call):
    print(f"🔧 Function Called - {function_call.name}")
    function_name = function_call.name
    args = function_call.args

    if function_name == "get_live_ticket_prices":
        origin = args.get("origin")
        destination = args.get("destination")
        departure_date = args.get("departure_date")
        return_date = args.get("return_date") or None

        return get_live_ticket_prices(origin, destination, departure_date, return_date)

    elif function_name == "book_flight":
        origin = args.get("origin")
        destination = args.get("destination")
        departure_date = args.get("departure_date")
        return_date = args.get("return_date") or None
        airline = args.get("airline", "Selected Airline")
        passenger_name = args.get("passenger_name", "Guest")

        return book_flight(origin, destination, departure_date, return_date, airline, passenger_name)

    else:
        return f"❌ Unknown function: {function_name}"


In [ ]:
def chat(message, history):
    full_message_history = []
    city_name = None

    # Convert previous history to Gemini-compatible format
    for h in history:
        if h["role"] == "user":
            full_message_history.append(
                types.Content(role="user", parts=[types.Part.from_text(text=h["content"])])
            )
        elif h["role"] == "assistant":
            full_message_history.append(
                types.Content(role="model", parts=[types.Part.from_text(text=h["content"])])
            )

    # Add current user message
    full_message_history.append(
        types.Content(role="user", parts=[types.Part.from_text(text=message)])
    )

    # Send to Gemini with tool config
    response = client.models.generate_content(
        model=MODEL_GEMINI,
        contents=full_message_history,
        config=generate_content_config
    )

    candidate = response.candidates[0]
    part = candidate.content.parts[0]
    function_call = getattr(part, "function_call", None)

    # Case: Tool call required
    if function_call:
        # Append model message that triggered tool call
        full_message_history.append(
            types.Content(role="model", parts=candidate.content.parts)
        )

        # Execute the tool
        tool_output = handle_tool_call(function_call)

        # Wrap and append tool output
        tool_response_part = types.Part.from_function_response(
            name=function_call.name,
            response={"result": tool_output}
        )
        
        full_message_history.append(
            types.Content(role="function", parts=[tool_response_part])
        )


        if function_call.name == "book_flight":
            city_name = function_call.args.get("destination").lower()
            

        # Send follow-up message including tool result
        followup_response = client.models.generate_content(
            model=MODEL_GEMINI,
            contents=full_message_history,
            config=generate_content_config
        )

        final_text = followup_response.text
        
        full_message_history.append(
            types.Content(role="model", parts=[types.Part.from_text(text=final_text)])
        )

        return final_text,city_name, history + [{"role": "assistant", "content": final_text}]
    else:
        text = response.text
        return text, city_name, history + [{"role": "assistant", "content": text}]


In [ ]:
def user_submit(user_input, history):
    history = history or []
    history.append({"role": "user", "content": user_input})
    
    response_text, city_to_image, updated_history = chat(user_input, history)

    # Speak the response
    try:
        talk(response_text)
    except Exception as e:
        print("[Speech Error] Speech skipped due to quota limit.")

    image = fetch_image(city_to_image) if city_to_image else None

    return "", updated_history, image, updated_history


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## ✈️ FlyJumbo Airline Assistant")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="Assistant", height=500, type="messages")
            msg = gr.Textbox(placeholder="Ask about flights...", show_label=False)
            send_btn = gr.Button("Send")

        with gr.Column(scale=2):
            image_output = gr.Image(label="Trip Visual", visible=True, height=500)

    state = gr.State([])
    
    send_btn.click(fn=user_submit, inputs=[msg, state], outputs=[msg, chatbot, image_output, state])
    msg.submit(fn=user_submit, inputs=[msg, state], outputs=[msg, chatbot, image_output, state])

demo.launch(inbrowser=True)